Código disponível [na minha página do Github](https://github.com/arthurkenzo/atividades_ia525)

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import mosek
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import networkx as nx
import random


from typing import Tuple
from itertools import product

• As aulas do curso são realizadas de segunda-feira a quinta-feira, das 19h às 23h.

• Duas turmas são conduzidas em paralelo ao longo do curso em cada ano.

• Em cada turma, os horários são fixos: as aulas de cada disciplina ocorrem sempre no mesmo dia da semana
para uma mesma turma. Note que turmas diferentes podem ter a mesma disciplina em dias diferentes.

• Em cada turma, cada dia de aula está associado a uma das disciplinas do curso: matemática, português, quı́mica
e fı́sica. Cada turma deve ter uma aula de cada disciplina.

• Cada dia de aula está dividido em dois blocos: aula teórica (19h-21h) e aula de exercı́cios (21h-23h).

• Os voluntários são selecionados por meio de processo especı́fico e são distribuı́dos em três papeis:
– Professor de teoria: responsável pela aula teórica de cada disciplina.
– Professor de exercı́cios: responsável pela aula de exercı́cios de cada disciplina.
– Monitor: responsável por apoiar o professor de exercı́cios na sua aula.

• A equipe de cada disciplina conta com dois professores de exercı́cios, dois professores de teoria e monitores
que devem ser alocados nos horários e distribuı́dos entre as turmas. Para uma mesma disciplina, o número de
monitores distribuı́dos entre as turmas deve ser aproximadamente igual. Cada turma fica com um professor de
cada tipo.

• A equipe de português é menor do que as demais. Assim, nas aulas de português, as duas turmas são unidas
em uma sala. Neste caso, ambos os professores de teoria podem trabalhar como professor de exercı́cios ou
monitor no segundo horário de aula. Nas demais disciplinas, cada voluntário atua em, no máximo, uma turma.

\begin{align*}
    \text{Encontrar }   & \sum_{ij} c_{ij}x_{ij}\\
    \text{sujeito a }   & \sum_{i} x_{ij} = 1 \quad \forall  j  \hspace{20pt} \text{(Alocar apenas uma pessoa para cada tarefa)} \\
                        & \sum_{j} x_{ij} = 1 \quad \forall  i  \hspace{20pt} \text{(Alocar apenas uma tarefa para cada pessoa)} \\

                        & x_{ij}\in \mathbb{B} \quad \forall  i,j  \hspace{20pt} \text{(Variáveis de decisão binárias)} \\
\end{align*}

In [2]:
# loading dataset into a pandas dataframe
availab = pd.read_csv('dados_exato.csv')
availab.fillna(0, inplace=True)
availab.index = [f'{i}' for i in range(len(availab))]

pd.set_option('display.expand_frame_repr', False) 
print("Pre-processed DataFrame:\n", availab.head())


Pre-processed DataFrame:
   area     cargo  seg19  seg21  ter19  ter21  qua19  qua21  qui19  qui21
0  mat  P Teoria    0.0    1.0    0.0    1.0    1.0    0.0    1.0    0.0
1  mat   Monitor    0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0
2  mat   Monitor    0.0    1.0    0.0    1.0    0.0    0.0    1.0    0.0
3  mat   Monitor    0.0    0.0    1.0    1.0    0.0    0.0    1.0    1.0
4  mat   Monitor    0.0    0.0    0.0    0.0    1.0    1.0    0.0    0.0


In [3]:
subjects = ['port', 'mat', 'fis', 'quim']
days = ['seg', 'ter', 'qua', 'qui']
shifts = ['19', '21']
voluntiers = [f'{i}' for i in range(len(availab))]

subjectIndexes = {}
for subject in subjects:
    subjectIndexes[subject] = availab[availab['area'] == subject].index.tolist()


In [4]:
nbVars = 0
# generate dictionary for indexing decision variables
alocationVars = {}
for voluntier in voluntiers:
    alocationVars[voluntier] = {}
    for day in days:
        alocationVars[voluntier][day] = {}
        for shift in shifts:
            # generate variables only for feasible hours based on availability table
            if availab.loc[voluntier, day+shift] == 1:
                alocationVars[voluntier][day][shift] = cp.Variable(boolean=True, name=f'aloc_{voluntier}_{day}_{shift}')
                nbVars += 1

subjectIndicators = {}
for subject in subjects:
    subjectIndicators[subject] = {}
    for day in days:
        subjectIndicators[subject][day] = cp.Variable(boolean=True, name=f'indic_{subject}_{day}')

print(f"Variáveis geradas: {nbVars}")


Variáveis geradas: 191


In [19]:
constraints = []

# only one shift per voluntier
for voluntier in voluntiers:
    # list of all days and shifts where the current voluntier is available
    shiftsAvailable = [alocationVars[voluntier][day][shift] 
                        for day in alocationVars.get(voluntier, {}) 
                        for shift in alocationVars[voluntier].get(day, {})]
    if shiftsAvailable:
        constraints += [cp.sum(shiftsAvailable) == 1]

# only one voluntier per shift
for day in days:
    for shift in shifts:
        # list of all voluntiers having a variable associated to the current day and shift
        voluntiersAvailable = [alocationVars[voluntier][day][shift] 
                               for voluntier in alocationVars 
                               if day in alocationVars.get(voluntier, {})
                                and shift in alocationVars[voluntier].get(day, {})]
        if voluntiersAvailable:
            constraints += [cp.sum(voluntiersAvailable) == 1]

# only one subject per day
for subject in subjects:
    constraints += [cp.sum([subjectIndicators[subject][day] for day in days])]

# alocate only voluntiers for the correct subject
for subject in subjects:
    for day in days:

        notAllowedVoluntiers = [index for _subject in subjectIndexes 
                                if _subject != subject 
                                for index in subjectIndexes[_subject] ]
        
        constraints += [subjectIndicators[subject][day] + cp.sum(notAllowedVoluntiers) == 1]

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [17]:
for subject in subjects:
    all_volunteers = set().union(*subjectIndexes.values())

    # Remove indexes for excluded subject
    # filtered_indexes = list(all_volunteers - set(subjectIndexes.get(subject, [])))
    filtered_indexes = [
    idx 
    for _subject in subjectIndexes 
    if _subject != subject 
    for idx in subjectIndexes[_subject]  # Keeps duplicates if they exist
]
    print(filtered_indexes)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '21', '22', '23', '24', '25', '26', '27', '28', '29']
['17', '18', '19', '20', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '21', '22', '23', '24', '25', '26', '27', '28', '29']
['17', '18', '19', '20', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '21', '22', '23', '24', '25', '26', '27', '28', '29']
['17', '18', '19', '20', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42']
